In [1]:
#@title Step 1: Download necessary files from our [GitHub page](https://github.com/cmb-chula/fetal-doppler-percentile)
#@markdown Hit the Play button.

#@markdown Please follow the instruction if there is an "Error" message

### clone github
!rm -rf fetal-doppler-percentile
!git clone https://github.com/cmb-chula/fetal-doppler-percentile

### check file integrity
import hashlib

data_md5 = hashlib.md5(open('fetal-doppler-percentile/data_stats.csv', 'rb').read()).hexdigest()
model_md5 = hashlib.md5(open('fetal-doppler-percentile/linear_models.pkl', 'rb').read()).hexdigest()

failed = False

if not data_md5 == '81170d2de5734d1788a35d3a6152ec95':
    print('Error: data_stats.csv may not have been properly downloaded! Please hit the Play button to download again')
    failed = True

if not model_md5 == '085ed697caf4c573693f9a3c9ca87c5d':
    print('Error: linear_models.pkl may not have been properly downloaded! Please hit the Play button to download again')
    failed = True

if not failed:
    print('')
    print('**********************')
    print('All files check out. Please proceed to the next code block')

Cloning into 'fetal-doppler-percentile'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.

**********************
All files check out. Please proceed to the next code block


In [17]:
#@title Step 2: Setup Python library and necessary codes
#@markdown Hit the play button and proceed

### install scikit-learn 1.2.0 to maintain compatability with trained models
!pip install --upgrade scikit-learn==1.2.0

import pickle
import pandas as pd
import numpy as np

from ipywidgets import HBox, VBox, Label, IntSlider, FloatSlider, Text, Checkbox, Layout, Button

active_slider_style = {'handle_color': '#FF7F50'}
inactive_slider_style = {'handle_color': '#999999'}

### load data into Python
models = pickle.load(open('fetal-doppler-percentile/linear_models.pkl', 'rb'))
all_params = sorted(models.keys())
data_stats = pd.read_csv('fetal-doppler-percentile/data_stats.csv', index_col = 0)

### create input user interface
input_header1 = Label(value = '[Step 1] Enter gestation age (GA), and if necessary, fetal heart rate (FHR)')
input_header2 = Label(value = 'Parameters with *** require FHR as input')
ga_w_label = Label(value = 'GA (week):  ')
ga_d_label = Label(value = 'GA (day):  ')

ga_w_slider = IntSlider(value = 30, min = 20, max = 40,
                        style = active_slider_style,
                        step = 1, continuous_update = False, readout = True, 
                        readout_format = 'd', indent = False)
ga_d_slider = IntSlider(value = 0, min = 0, max = 6,
                        style = active_slider_style, 
                        step = 1, continuous_update = False, readout = True, 
                        readout_format = 'd', indent = False)

fhr_label = Label(value = 'FHR (bpm):  ')
fhr_slider = IntSlider(value = 145, min = 80, max = 200,
                       style = active_slider_style,
                       step = 1, continuous_update = False, readout = True, 
                       readout_format = 'd', indent = False)

input_labels = VBox([ga_w_label, ga_d_label, fhr_label], layout = Layout(width = '120px', align_items = 'flex-end'))
input_sliders = VBox([ga_w_slider, ga_d_slider, fhr_slider], layout = Layout(width = '350px'))
input_ui = HBox([input_labels, input_sliders])
input_ui = VBox([input_header1, input_header2, input_ui])

### create output user interface
labels = {}
sliders = {}
checkboxes = {}
textfields = {}

slide_sd_range = 5

for p in all_params:
    mean_p = data_stats.loc[p, 'Mean']
    sd_p = data_stats.loc[p, 'SD']
    
    if not pd.isna(data_stats.loc[p, 'Unit']):
        description = p + ' (' + data_stats.loc[p, 'Unit'] + ')'
    else:
        description = p
        
    if data_stats.loc[p, 'Info'] == 'GA+FHR':
        description = '***' + description
    
    labels[p] = Label(value = description + ': ')
    sliders[p] = FloatSlider(value = mean_p, min = max(0, mean_p - slide_sd_range * sd_p), 
                             style = inactive_slider_style,
                             max = mean_p + slide_sd_range * sd_p, disabled = True,
                             step = 0.01, continuous_update = False, readout = True, 
                             readout_format = '.2f', indent = False)
    
    checkboxes[p] = Checkbox(value = False, indent = False)
    checkboxes[p].layout.width = '20px'
    checkboxes[p].name = p ## Use name to define relationship to other UI elements
    
    textfields[p] = Text(value = '', disabled = True, indent = False)
    textfields[p].layout.width = '100px'

output_header1 = Label(value = '[Step 2] Select parameter(s) of interest, enter their values, and hit button to process')
output_header2 = Label(value = 'Parameter value can be set via the slider or manually by clicking on the number')
run_button = Button(description = 'Predict', tooltip = 'Click to get estimated percentiles')

output_labels = VBox([labels[p] for p in all_params], layout = Layout(width = '120px', align_items = 'flex-end'))
output_sliders = VBox([sliders[p] for p in all_params], layout = Layout(width = '350px'))

output_cb_labels = VBox([Label(value = 'Selected: ') for p in all_params], layout = Layout(width = '70px'))
output_cbs = VBox([checkboxes[p] for p in all_params], layout = Layout(width = '50px'))

output_tf_labels = VBox([Label(value = 'Percentile: ') for p in all_params], layout = Layout(width = '70px'))
output_tfs = VBox([textfields[p] for p in all_params], layout = Layout(width = '150px'))

output_ui = HBox([output_labels, output_sliders, output_cb_labels, output_cbs, output_tf_labels, output_tfs])
output_ui = VBox([output_header1, output_header2, output_ui, run_button])

### define interaction between user interface elements
def toggle_slider(sender): ## from checkboxes
    p = sender.owner.name
    
    if sender.owner.value == True:
        sliders[p].disabled = False
        sliders[p].style = active_slider_style
    else:
        sliders[p].disabled = True
        sliders[p].style = inactive_slider_style
        textfields[p].value = ''

def process_input(): ## GA, Log GA, GA^2, FHR, Log FHR, FHR^2
    ga = float(ga_w_slider.value) + float(ga_d_slider.value) / 7
    ga_log = np.log(ga)
    ga2 = ga ** 2
    
    fhr = float(fhr_slider.value)
    fhr_log = np.log(fhr)
    fhr2 = fhr ** 2
    
    ga_std = (ga - data_stats.loc['GA', 'Mean']) / data_stats.loc['GA', 'SD']
    ga_log_std = (ga_log - data_stats.loc['log GA', 'Mean']) / data_stats.loc['log GA', 'SD']
    ga2_std = (ga2 - data_stats.loc['GA^2', 'Mean']) / data_stats.loc['GA^2', 'SD']
    
    fhr_std = (fhr - data_stats.loc['FHR', 'Mean']) / data_stats.loc['FHR', 'SD']
    fhr_log_std = (fhr_log - data_stats.loc['log FHR', 'Mean']) / data_stats.loc['log FHR', 'SD']
    fhr2_std = (fhr2 - data_stats.loc['FHR^2', 'Mean']) / data_stats.loc['FHR^2', 'SD']
    
    return pd.DataFrame([[ga_std, ga_log_std, ga2_std, fhr_std, fhr_log_std, fhr2_std]], index = ['0'],
                        columns = ['GA', 'log GA', 'GA^2', 'FHR', 'log FHR', 'FHR^2'])
        
def predict_param(p, input_std):
    measurement = (sliders[p].value - data_stats.loc[p, 'Mean']) / data_stats.loc[p, 'SD']
    
    if data_stats.loc[p, 'Info'] == 'GA+FHR':
        input_merged = input_std.copy()
    else:
        input_merged = input_std.loc[:, ['GA', 'log GA', 'GA^2']].copy()
    
    input_merged[p] = [measurement]
    prediction = models[p].predict(input_merged)
    
    return prediction
    
def predict_all(sender): ## from button
    input_std = process_input()
    
    for p in all_params:
        if checkboxes[p].value == True:
            prediction = predict_param(p, input_std)[0] * 100
            
            if prediction < 0:
                prediction = 0.01
            elif prediction > 100:
                prediction = 99.99
            
            textfields[p].value = str(prediction)[:5]

### link user interface elements to interaction
for p in all_params:
    checkboxes[p].observe(toggle_slider)
    
run_button.on_click(predict_all)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
#@title Step 3: Launch the predictor interface
#@markdown Hit the run button and follow the instruction on the user interface
main_ui = VBox([input_ui, output_ui])
display(main_ui)